In [1]:

import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import os

DATA_PATH = "data/nifty_data.csv"
MODEL_PATH = "models/nifty_lstm_model.h5"
TICKER = "^NSEI"      # NIFTY 50 index symbol
EPOCHS = 25
SEQ_LEN = 60

os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)

print("Downloading NIFTY 50 data...")
data = yf.download(TICKER, start="2015-01-01", end=None)
data.to_csv(DATA_PATH)
print(f"Data saved to {DATA_PATH}")

df = data[["Close"]].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

X_train, y_train = [], []
for i in range(SEQ_LEN, len(scaled_data)):
    X_train.append(scaled_data[i-SEQ_LEN:i, 0])
    y_train.append(scaled_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print("Training data shape:", X_train.shape)

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

print("Training LSTM model...")
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=32, verbose=1)

model.save(MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")

train_pred = model.predict(X_train)
train_pred_inv = scaler.inverse_transform(train_pred)
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))

mse = tf.keras.losses.MeanSquaredError()(y_train_inv, train_pred_inv).numpy()
print(f"Training MSE: {mse:.2f}")


from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

print("Evaluating model performance...")

train_pred = model.predict(X_train)

train_pred_inv = scaler.inverse_transform(train_pred)
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))

mse = mean_squared_error(y_train_inv, train_pred_inv)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train_inv, train_pred_inv)
r2 = r2_score(y_train_inv, train_pred_inv)

mape = np.mean(np.abs((y_train_inv - train_pred_inv) / y_train_inv)) * 100
accuracy = 100 - mape

print("\nModel Performance on Training Data:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Approx. Training Accuracy: {accuracy:.2f}%")


/var/folders/wg/d1y3qvk103gdr7sntsqgtyb40000gn/T/ipykernel_3408/2570951602.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(TICKER, start="2015-01-01", end=None)
[*********************100%***********************]  1 of 1 completed

Data saved to data/nifty_data.csv
Training data shape: (2626, 60, 1)



/Users/bdeepaksai/Library/Caches/JetBrains/PyCharmCE2024.2/demo/PyCharmLearningProject/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,033 (203.25 KB)

 Trainable params: 52,033 (203.25 KB)

 Non-trainable params: 0 (0.00 B)

Training LSTM model...
Epoch 1/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0134
Epoch 2/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0017
Epoch 3/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0015
Epoch 4/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0014
Epoch 5/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0014
Epoch 6/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0014
Epoch 7/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0011
Epoch 8/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0011
Epoch 9/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0010
Epoch 10/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 9.8919e-04
Epoch 11/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 9.8664e-04
Epoch 12/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 9.2887e-04
Epoch 13/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 9.0418e-04
Epoch 14/25
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 7.9506e-04
Epoch 15/25
83/83 ━━━━━━━━━━

Model saved to models/nifty_lstm_model.h5
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
Training MSE: 104466.38
Evaluating model performance...
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step

Model Performance on Training Data:
Mean Squared Error (MSE): 104466.3785
Root Mean Squared Error (RMSE): 323.2126
Mean Absolute Error (MAE): 232.4289
R² Score: 0.9967
Mean Absolute Percentage Error (MAPE): 1.58%
Approx. Training Accuracy: 98.42%
